In [47]:
import pandas as pd
import numpy as np
import os

In [26]:
def transform(row):
    # transform the large amount of data in row to a much smaller representative subset

    # extract important data from row
    scan_1080 = row.iloc[0:1080]
    local_goal = row.iloc[1084:1088]
    robot_pose = row.iloc[1088:1092]
    cmd = row.iloc[1092:1094]

    # reduce scan data
    # 1) bin data using averages from 1080 to 135 -> reduce by factor of 8
    # 2) bin data using min from 135 to 9 -> reduce by factor of 15
    scan = []  # initalize transformed scan array
    for i in range(9):  # there wil be 9 values in scan array
        bin = []  # initalize the bin which represents a 30 degree slice of scan
        for j in range(15):  # there will be 15 values in bin array
            bin.append(np.average(scan_1080[8*j+120*i:8*j+120*i+8]))  # take the average value over 8 sequential scan data points and add it to bin
        scan.append(min(bin))  # add the smallest bin value to scan array

    # compute difference in local goal and pose
    delta = list(np.array(local_goal) - np.array(robot_pose))

    new_row = scan + delta + list(np.array(cmd))

    return new_row

In [45]:
def find_csv_filenames( path_to_dir, suffix=".csv" ):
    filenames = os.listdir(path_to_dir)
    return [ filename for filename in filenames if filename.endswith( suffix ) ]

In [43]:
path_old = 'Training/corridor_CSV/'
filenames = find_csv_filenames(path_old)

path_new = 'Training_Clean/corridor_CSV/'
path_new = os.path.join(os.getcwd(), path_new)
if not os.path.exists(path_new):
   os.makedirs(path_new)


In [55]:
# transform all csvs in the path_old directory and put them in the path_new directory
for f, name in enumerate(filenames):
    print("Reading {0}".format(f+1))
    name = os.path.splitext(name)[0]
    df_old = pd.read_csv(path_old + name + '.csv', header=None)
    df_new = pd.DataFrame(columns = ["-135:-105", "-105:-75", "-75:-45", "-45:-15", "-15:+15", "15:45", "45:75", "75:105", "105:135", "dx", "dy", "dqk", "dqr", "v", "w"])
    print("Processing {0}".format(f+1))
    
    for index, row in df_old.iterrows():
        df_new.loc[len(df_new)] = transform(row)
    df_new.to_csv(path_new + name + '_clean.csv', index=False)
    break

Reading 1
Processing 1
